In [3]:
import pandas as pd
import vectorbtpro as vbt
from datetime import datetime, timedelta
import pytz
import numpy as np
import pandas_ta as ta
import logging
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply
logging.basicConfig(level=logging.WARNING, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
import multiprocessing as mp
mp.set_start_method('fork')


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [4]:

start = "2018-01-01"
# Enter your parameters here
metric = 'total_return'

start_date = datetime(2020, 1, 1, tzinfo=pytz.utc)  # time period for analysis, must be timezone-aware
end_date = datetime.now(pytz.utc)
# end_date = datetime(2020, 1, 1, tzinfo=pytz.utc)

# The following is the number of days to look back for the analysis
time_buffer = timedelta(days=100)  # buffer before to pre-calculate SMA/EMA, best to set to max window
freq = '1h'

vbt.settings.portfolio['init_cash'] = 100_000.  # 100,000$
# vbt.settings.portfolio['fees'] = 0.0025  # 0.25%
# vbt.settings.portfolio['slippage'] = 0.0025  # 0.25%

# get binance data doing it this way allows for you to update your data rather than re-downloading it
# binance_data = vbt.BinanceData.fetch(symbols,timeframe=freq, start=start_date,end="now UTC")
# binance_data.save("binance_data.pkl")

# If you already have the data downloaded, you can load it
# binance_data = vbt.BinanceData.load("binance_data.pkl")
# binance_data = binance_data.update() if you want to update it.
futures_path = '/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m_futures.pkl'
# You can grab the necessary file from this google drive link. https://drive.google.com/drive/folders/1jKy2DMbBow-J5jTvPutw-j17m7Ss8R3i?usp=drive_link

If you don't have the datafile yet you can grab it from this [link](https://drive.google.com/drive/folders/1jKy2DMbBow-J5jTvPutw-j17m7Ss8R3i?usp=drive_link)

In [5]:
btc = vbt.BinanceData.load(futures_path)
df = btc.get()

In [6]:
# create a signal array the same length as the dataframe
signal = np.zeros(len(df))
signal = pd.Series(signal, index=df.index)
# set the signal to 1 at midnight UTC
signal[df.at_time('00:00').index] = 1
# Create a -1 right before the signal
signal[df.at_time('12:00').index] = -1 # random short signals at noon
df['signal'] = signal
# Convert to int
df['signal'] = df['signal'].astype(int)
df['signal'].sum()

0

In [28]:
# This will be handy when we want to plot so it doesn't store all the html files in the root


def create_filename(stats_df):
    params = stats_df._strategy.__dict__['_params']
    filename = 'html_files/'
    # unpack the best params and add them to a filename
    for key, value in params.items():
        # keep the first 5 letters from each key
        key = key[:5]
        # just keep the first 4 digits from each value
        value= str(value)[:4]
        filename += f'{key}_{value}_'
    filename += '.html'
    return filename

In [31]:
class LONG_SHORT_Underwater_w_decay_and_deleverage(Strategy):
    # All of the following variables can be used during optimization
    initial_position_size = 0.3
    percent_invested_threshold = 0.3
    atr_length = 14  # 14 days
    atr_multiplier = 0.5
    add_size = 0.1
    delay_period = 1000
    delta_time = 1000
    upper_bound_profit_target = 0.05
    lower_bound_loss_threshold = 0.02
    take_profit_loss_reduction = -0.1 # amount take profit is reduced by if the position is highly leveraged and we wish to trim
    deleverage_pct = 0.30  # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim
    bounce_multiplier = 1.5  # multiply the ATR Treshold by this amount to get the bounce threshold we are looking to bounce off a low "max pain" point before we reduce our position
    max_loss_threshold = -0.05 # if the position is down this much, and we are fully invested we will reduce our positio
    max_hold_length = None # Number of minutes to hold a position before closing it None if you want to exit naturally
    
    def SIGNAL(self):
        return self.data.signal

    def ATR(self, df, length):
        return df.ta.atr(length=length)

    def bars_since_first_trade(self, use_for_indexing=False):
        """
        Calculate the number of bars since the first trade was entered.
        If use for indexing is true then it will return 1 if there are no trades.
        This way you can retrieve the last element of a list by applying a - to the return value.
        eg. self.equity[-self.bars_since_first_trade(use_for_indexing=True):] gives you the array of your account's equity since you entered the first trade that is currently open
        """
        if len(self.trades) > 0:
            self.first_trade_entry_bar = self.trades[0].entry_bar
            bars_since_first_trade = len(self.data.Close) - self.first_trade_entry_bar
            return bars_since_first_trade
        elif use_for_indexing:
            return 1
        else:
            return 0

    def custom_decay_func(
        self,
        x,
        delay_period,
        upper_bound_profit_target,
        lower_bound_loss_threshold,
        delta_time,
    ):
        """
        This function is used to calculate the decayed take profit x represents the number of bars since the first trade was entered
        x is the number of bars since the first trade was entered
        delay_period is the number of bars to wait before starting to decay
        upper_bound_profit_target is the upper bound of the take profit
        lower_bound_loss_threshold is the lower bound of the take profit
        delta_time is the number of bars over which to transition from the upper bound to the lower bound
        """
        if x <= delay_period:
            return upper_bound_profit_target
        elif delay_period < x < delay_period + delta_time:
            # Calculate the x value for the cos function
            transition_x = (x - delay_period) / delta_time * np.pi
            # Calculate the decayed take profit
            return (-np.cos(transition_x) + 1) / 2 * (
                lower_bound_loss_threshold - upper_bound_profit_target
            ) + upper_bound_profit_target
        else:
            return lower_bound_loss_threshold

    def init(self):
        super().init()
        self.signal = self.I(self.SIGNAL)
        self.atr = self.I(self.ATR, self.data.df, self.atr_length)
        self.daily_atr = resample_apply("1D", self.ATR, self.data.df, length=14)
        count_NaN = len(self.atr) - len(
            pd.Series(self.atr).dropna()
        )  # This is used for the progress bar
        self.length_of_data = (
            len(self.data.Close) - count_NaN
        )  # This is used for the progress bar
        self.equity_during_trade = []  # Keeps a list for the equity during the trade
        self.long_short_flag = (
            None  # This is used to keep track of whether we are long or short
        )
        self.price_at_last_trim = (
            0  # This is used to keep track of the price at the last trim
        )

    def next(self):
        super().next()

        price = self.data.Close[-1]
        position_value = self.position.size * price
        percent_invested = (
            position_value / self.equity
        )  # this will come in handy if we decide to change behavior once XX% is invested
        atr_threshold_pct = (
            self.atr_multiplier * self.daily_atr[-1] / price
        )  # This is the ATR threshold times a multiplier calculated as a percentage of price
        bars_since_trade_open = self.bars_since_first_trade(use_for_indexing=True)
        highly_leveraged = abs(percent_invested) > self.percent_invested_threshold
        # Calculate the decayed take profit
        decayed_take_profit = self.custom_decay_func(
            bars_since_trade_open,
            self.delay_period,
            self.upper_bound_profit_target,
            self.lower_bound_loss_threshold,
            self.delta_time,
        )

        # Calculate the lowest the equity ever got since we have been in a trade
        if self.position:  
            self.lowest_equity_during_trade = min(self.lowest_equity_during_trade, self.equity)
        else:
            self.lowest_equity_during_trade = float('inf')  # Reset to a very high number when not in a position

        low_point_in_trade = min(self.equity_during_trade, default=0)

        # Calculate the percentage bounce from the low point in the trade this is based on equity so works for long or short
        bounce_from_low_pct = (lambda x: (self.equity - x) / x if x != 0 else 0)(
            low_point_in_trade
        )  # The lambda function is used to avoid a divide by zero error

        # Opening a Trade on a signal from the LSTM
        if not self.position:
            if self.signal == 1:
                self.buy(size=self.initial_position_size)
                self.long_short_flag = 1
            elif self.signal == -1:
                self.sell(size=self.initial_position_size)
                self.long_short_flag = -1
        
        # If we are in a short trade and the account is fully invested and the loss is greater than the max loss threshold then close the trade
        if self.long_short_flag == -1 and (abs(percent_invested) > 1 and self.position.pl_pct < self.max_loss_threshold):
            self.position.close() # If you want a partial close add `portion=self.deleverage_pct`
        
        # If we are in a trade and it meets our profit criteria then close the trade
        if self.position.pl_pct > decayed_take_profit:
            self.position.close()
            self.price_at_last_trim = 0
            # print(f'Closing at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')

        # If we are in a trade and it meets our loss criteria then close a portion of the trade on a bounce from the low point
        if self.position and self.position.pl_pct < -atr_threshold_pct:
            # Check to see if we are also down on our last trade
            if self.trades[-1].pl_pct < -atr_threshold_pct:
                if self.long_short_flag == 1:
                    self.buy(size=self.add_size)
                elif self.long_short_flag == -1:
                    self.sell(size=self.add_size)
        
        # If we are totally upside down then reduce the position size
        if self.position and self.position.pl_pct < decayed_take_profit + self.take_profit_loss_reduction:
            if self.trades[-1].pl_pct < -atr_threshold_pct: # if we are down more than 1 ATR threshold then reduce the position size and take a tiny loss
                self.position.close(portion=self.deleverage_pct)
                print(f'Closing at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')
        
        # Deleverage if we are over a certain percent invested and the market is recovering from the low point
        if (
            highly_leveraged
            and bounce_from_low_pct > self.bounce_multiplier * atr_threshold_pct
        ):

            if (self.position.pl_pct > decayed_take_profit + self.take_profit_loss_reduction):
                if (self.long_short_flag == 1) and (self.price_at_last_trim == 0 or price > self.price_at_last_trim *(1 + 2 * atr_threshold_pct)):
                    self.position.close(portion=self.deleverage_pct)
                    # Keep track of the price to avoid trimming too often
                    self.price_at_last_trim = price 
                    # print(f'Deleveraging at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')
                elif (self.long_short_flag) == -1 and (self.price_at_last_trim == 0 or price < self.price_at_last_trim *(1 - 0.5 * atr_threshold_pct)):
                    self.position.close(portion=self.deleverage_pct)
                    # Keep track of the price to avoid trimming too often
                    self.price_at_last_trim = price 
                    # print(f'Deleveraging at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')



start = "2022-01-01"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = "2023-06-03"  # It will always close any open trades at the end of the backtest
bt = Backtest(
    df.loc[start:end],
    LONG_SHORT_Underwater_w_decay_and_deleverage,
    cash=100_000_000,
    exclusive_orders=False,
    trade_on_close=True,
    margin=1,
)
stats = bt.run()
filename = create_filename(stats)
print(stats)
bt.plot(resample=False, filename=filename)

Closing at 37490.0 at 2022-01-21 21:34:00+00:00 Position PNL is -2953368.540000002
Closing at 36274.45 at 2022-01-21 21:50:00+00:00 Position PNL is -3222216.520000004
Closing at 35032.34 at 2022-01-22 09:41:00+00:00 Position PNL is -3331345.620000003
Closing at 33802.27 at 2022-01-24 10:12:00+00:00 Position PNL is -3344295.0500000017
Closing at 41054.46 at 2022-02-04 23:46:00+00:00 Position PNL is -4675078.159999998
Closing at 42155.0 at 2022-02-06 14:31:00+00:00 Position PNL is -4428986.119999999
Closing at 43220.0 at 2022-02-07 14:41:00+00:00 Position PNL is -4048603.7799999993
Closing at 44300.47 at 2022-02-07 18:02:00+00:00 Position PNL is -3670584.2800000003
Closing at 45456.38 at 2022-02-10 15:41:00+00:00 Position PNL is -3380556.579999998
Closing at 36200.0 at 2022-02-24 02:56:00+00:00 Position PNL is -4131866.6999999997
Closing at 35073.5 at 2022-02-24 03:40:00+00:00 Position PNL is -4020097.0
Closing at 39752.8 at 2022-03-04 19:58:00+00:00 Position PNL is -2095162.9999999956
C

GridPlot(id='p19341', ...)

In [29]:
filename = create_filename(stats)

In [30]:
stats
print(stats)
bt.plot(resample=False, filename=filename)

Start                     2022-01-01 00:00...
End                       2023-06-03 23:12...
Duration                    518 days 23:12:00
Exposure Time [%]                   91.453648
Equity Final [$]                  111616681.5
Equity Peak [$]                   111841269.5
Return [%]                          11.616682
Buy & Hold Return [%]              -41.491126
Return (Ann.) [%]                    8.035556
Volatility (Ann.) [%]               23.318242
Sharpe Ratio                         0.344604
Sortino Ratio                        0.490093
Calmar Ratio                         0.388506
Max. Drawdown [%]                  -20.683231
Avg. Drawdown [%]                   -0.421241
Max. Drawdown Duration      406 days 05:49:00
Avg. Drawdown Duration        3 days 01:40:00
# Trades                                  898
Win Rate [%]                        23.385301
Best Trade [%]                      12.810546
Worst Trade [%]                    -43.835413
Avg. Trade [%]                    

GridPlot(id='p17944', ...)

In [8]:
bt.plot(resample=False)

GridPlot(id='p3118', ...)

## Now let's optimize the long short strategy
Note, this will take a while to run depending on how many parameters you want to optimize on.

I'm copying the class down here to remove the progress bars they conflict with the optimizer progress bars

In [18]:
stats, heatmap = bt.optimize(
    # initial_position_size = np.arange(0.3, 0.6, 0.1).tolist(),
    # percent_invested_threshold = 0.7 
    # atr_length = np.arange(7,37,10).to_list(), # 14 days
    atr_multiplier = np.arange(0.3, 1.5, 0.3).tolist(),
    # add_size = np.arange(0.10,0.30, 0.10).tolist(),
    # delay_period = np.arange(250,1500,500).tolist(),
    # delta_time = np.arange(1000,5000,1000).tolist(),
    upper_bound_profit_target = np.arange(0.03, 0.10, 0.02).tolist(),
    # lower_bound_loss_threshold = np.arange(-0.10, 0.03, 0.03).tolist(),
    take_profit_loss_reduction = np.arange(-0.15, -0.05, 0.05).tolist(), # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    # deleverage_pct = 0.5 # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim
    max_loss_threshold = np.arange(-0.05, -0.20, -0.05).tolist(),
    maximize='Equity Final [$]', 
    # maximize='Max. Drawdown [%]',# this can be any of the column names from the stats table the output of the backtest
    max_tries=200,
    random_state=0,
    return_heatmap=True)
best_params = stats._strategy.__dict__["_params"] # This will print out all the parameters used for the best backtest
print(f'Best Parameters: {best_params}')
heatmap.sort_values(ascending=False).iloc[:-5] # print the top 5 parameter sets

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Best Parameters: {'atr_multiplier': 0.6, 'upper_bound_profit_target': 0.05, 'take_profit_loss_reduction': -0.15, 'max_loss_threshold': -0.05}


atr_multiplier  upper_bound_profit_target  take_profit_loss_reduction  max_loss_threshold
0.6             0.05                       -0.10                       -0.05                 1.395531e+08
                                           -0.15                       -0.05                 1.395531e+08
                                                                       -0.15                 1.395531e+08
                                                                       -0.20                 1.395531e+08
                                           -0.10                       -0.10                 1.395531e+08
                                                                                                 ...     
0.3             0.03                       -0.15                       -0.05                 9.287254e+07
                                                                       -0.10                 9.287254e+07
                0.09                       -0.15              

In [10]:
# Plot the heatmap
from backtesting.lib import plot_heatmaps
plot_heatmaps(heatmap, agg='mean')


GridPlot(id='p4208', ...)

### Rerun the stats for the best strategy


In [21]:
bt.run(**best_params)
filename = create_filename(stats)
bt.plot(resample=False, filename=filename)

KeyError: 'params'

In [17]:
filename = 'html_files/'
# unpack the best params and add them to a filename
for key, value in best_params.items():
    # keep the first 5 letters from each key
    key = key[:5]
    # just keep the first 4 digits from each value
    value= str(value)[:4]
    filename += f'{key}_{value}_'
filename += '.html'
print(filename)

bt.plot(filename=filename)

html_files/atr_m_1.2_upper_0.09_take__-0.1_max_l_-0.0_.html


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p13752', ...)